In [281]:
import torch
from torch import nn
torch.set_printoptions(precision=2, sci_mode=False)
torch.set_default_dtype(torch.float64)
import numpy as np
import torch.nn.functional as F
import math
from pathlib import Path
import torch.nn as nn


In [54]:
def load_tensors(filename):
    tensors_dict = {}
    
    # Tracking current tensor state
    current_name = "default_tensor"
    current_metadata = {}
    current_data = []
    
    meta_keys = ['size', 'ndim', 'shape', 'stride', 'elem_size', 'requires_grad']

    def finalize_tensor(name, meta, data):
        """Helper to reshape data and store in the dictionary."""
        if not data:
            return
        
        target_shape = [int(s) for s in meta.get('shape', [len(data)])]
        # Using float64 (double) as your images show high precision decimals
        # if name == current_name:
        #     tensors_dict = torch.tensor(data, dtype=torch.float32).reshape(target_shape)
        # else:
        tensors_dict[name] = torch.tensor(data, dtype=torch.float64).reshape(target_shape)

    with open(filename, 'r') as f:
        for line in f:
            # Split by comma and remove empty strings/whitespace
            parts = [p.strip() for p in line.split(',') if p.strip()]
            
            if not parts:
                continue

            label = parts[0].replace(':', '')

            # 1. Check if it's a known metadata key
            if label in meta_keys:
                vals = [float(v) for v in parts[1:]]
                current_metadata[label] = vals[0] if len(vals) == 1 else vals
            
            else:
                try:
                    # 2. Try to parse as numeric data
                    row_data = [float(p) for p in parts]
                    current_data.extend(row_data)
                except ValueError:
                    # 3. If it's a string but NOT metadata, it's a new tensor name
                    # Save the previous tensor first
                    if current_data:
                        finalize_tensor(current_name, current_metadata, current_data)
                    
                    # Reset for the new tensor
                    current_name = label
                    current_metadata = {}
                    current_data = []

    # Finalize the last tensor in the file
    finalize_tensor(current_name, current_metadata, current_data)
    if "default_tensor" in tensors_dict.keys():
        tensors_dict = tensors_dict["default_tensor"]
    return tensors_dict

In [55]:
def load_gpt_model(path):
    gpt_model = {}
    folder_path = Path(path)
    for file in folder_path.iterdir():
        if file.is_file():  # Ensure it's a file and not a subfolder
            filename = Path(file.name).stem
            # print(f"Filename: {filename}")
            gpt_model[filename] = load_tensors(file)
            # print(f"Full Path: {file}")
    return gpt_model

In [56]:
gpt_model_c = load_gpt_model("/Users/uonliaquat/workspace/zerograd/models")
gpt_model_c["transformer_layer_0__self_attention_layer_heads_proj"]

{'Weights': tensor([[ 0.85,  0.90, -0.04,  ..., -0.58, -0.18,  0.63],
         [ 0.28, -0.13, -0.90,  ..., -0.52,  0.49,  0.62],
         [ 0.62,  0.66, -0.76,  ...,  0.83,  0.33, -0.73],
         ...,
         [ 0.94,  0.39,  0.47,  ...,  0.83,  0.61, -0.22],
         [-0.61,  0.08, -0.45,  ...,  0.30, -0.30, -0.89],
         [-0.06,  0.91, -0.10,  ..., -0.90,  0.63, -0.89]]),
 'Output': tensor([[[    -2.76,     -0.48,      5.78,      0.43,      5.98,      6.62,
               -6.77,      1.45,      6.02,     -8.07,      5.96,     14.18,
               -1.58,      3.50,      8.39,      1.56,      2.96,    -13.13,
               -3.33,      4.71,      5.05,      7.32,      7.52,      9.71,
                4.27,    -10.34,      2.62,      6.44,    -10.13,     -6.80,
               -1.95,    -16.84],
          [    -0.42,     -5.68,     -5.70,     10.29,     12.53,      4.34,
                4.55,      1.88,      2.43,     -8.14,     -1.08,      5.87,
               10.10,     -5.06,    

In [57]:
# base_path = '../models'
# gpt_model_c = {
#     "input_tokens":  load_tensors(f'{base_path}/input_tokens.csv')['default_tensor'].long(),
#     "token_embeddings_layer": load_tensors(f'{base_path}/gpt_model.token_embed_layer.csv'),
#     "pos_embeddings_layer": load_tensors(f'{base_path}/gpt_model.pos_embed_layer.csv'),
#     "position_indicies":  load_tensors(f'{base_path}/gpt_model.workspace.position_indicies.csv')['default_tensor'].long(),
#     "input_embeddings": load_tensors(f'{base_path}/gpt_model.workspace.input_embeddings.csv')['default_tensor']
# }
# gpt_model_c

In [58]:
def tensors_within_tolerance(a, b, atol):
    if a.shape != b.shape:
        print("Shape mismatch:", a.shape, b.shape)
        return False

    max_diff = round((a - b).abs().max().item(), 2)
    print("max |diff|:", max_diff)

    return max_diff <= atol

## GPT

In [59]:
for key in gpt_model_c.keys():
    print(key)

transformer_layer_0__self_attention_layer_attention_scores_0
gpt_model.pos_embed_layer
transformer_layer_0__self_attention_layer_attention_scores_1
input_tokens
transformer_layer_0__self_attention_layer_context_vecs_0
transformer_layer_0__self_attention_layer_w_key
transformer_layer_0__self_attention_layer_context_vecs_1
transformer_layer_0__self_attention_layer_w_query
gpt_model.token_embed_layer
transformer_layer_0__self_attention_layer_attention_scores_scaled_0
gpt_model.workspace.position_indices
transformer_layer_0__self_attention_layer_attention_scores_scaled_1
transformer_layer_0__self_attention_layer_keys_transposed_0
transformer_layer_0__self_attention_layer_queries_chunks_1
transformer_layer_0__self_attention_layer_queries_chunks_0
transformer_layer_0__self_attention_layer_keys_transposed_1
transformer_layer_0__self_attention_layer_w_value
transformer_layer_0__self_attention_layer_keys_chunks_0
transformer_layer_0__self_attention_layer_keys_chunks_1
transformer_layer_0__self_

In [60]:
def find_key(target_dict, substring):
    """
    Returns the first key that contains the substring.
    Returns None if no match is found.
    """
    return next((k for k in target_dict if substring in k), None)

# Usage:

In [61]:
key_transposed_global = None
class SelfAttentionMultiHead(nn.Module):
    def __init__(self, self_attention_layer_c, n_heads, atol):
        super().__init__()
        
        self.n_heads = n_heads
        self.atol = atol
        self.self_attention_layer_c = self_attention_layer_c
        
        W_query_weights =   self_attention_layer_c[find_key(self_attention_layer_c, 'w_query')]['Weights']
        W_key_weights =     self_attention_layer_c[find_key(self_attention_layer_c, 'w_key')]['Weights']
        W_value_weights =   self_attention_layer_c[find_key(self_attention_layer_c, 'w_value')]['Weights']
        head_proj_weights =   self_attention_layer_c[find_key(self_attention_layer_c, 'heads_proj')]['Weights']

        # print("W_query_weights\n ", W_query_weights)
        W_query_weights = W_query_weights.t()
        W_key_weights = W_key_weights.t()
        W_value_weights = W_value_weights.t()
        head_proj_weights = head_proj_weights.t()

        self.W_query = nn.Linear(W_query_weights.shape[0], W_query_weights.shape[1], bias=False)
        self.W_query.weight = nn.Parameter(W_query_weights)

        self.W_key = nn.Linear(W_key_weights.shape[0], W_key_weights.shape[1], bias=False)
        self.W_key.weight = nn.Parameter(W_key_weights)

        self.W_value = nn.Linear(W_value_weights.shape[0], W_value_weights.shape[1], bias=False)
        self.W_value.weight = nn.Parameter(W_value_weights)

        self.heads_proj = nn.Linear(head_proj_weights.shape[0], head_proj_weights.shape[1], bias=False)
        self.heads_proj.weight = nn.Parameter(head_proj_weights)

    
    def forward(self, x):
        # print(f"Input Embeddings: \nShape: {x.shape}\n{x}")
        # print(f"W_Query\nShape: {self.W_query.weight.shape}\n{self.W_query.weight}")
        # print(f"W_Key\nShape:   {self.W_key.weight.shape}\n{self.W_key.weight}")
        # print(f"W_Value\nShape: {self.W_value.weight.shape}\n{self.W_value.weight}")
        # print(f"heads_proj\nShape: {self.heads_proj.weight.shape}\n{self.heads_proj.weight}")
        print("===========================================")

        query = self.W_query(x)
        key = self.W_key(x)
        value = self.W_value(x)
        self.layer_name = 'self_attention_layer'
        query_matched = tensors_within_tolerance(query, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_w_query')]['Output'], self.atol)
        key_matched = tensors_within_tolerance(key,  self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_w_key')]['Output'], self.atol)
        value_matched = tensors_within_tolerance(value,  self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_w_value')]['Output'], self.atol)

        print(f"query_matched:   {query_matched}")
        print(f"key_matched:     {key_matched}")
        print(f"value_matched:   {value_matched}")
        # print("Python", query)
        # print("C", self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_w_query')]['Output'])

        queries_chnuks  = torch.chunk(query, self.n_heads, -1)
        keys_chnuks     = torch.chunk(key , self.n_heads, -1)
        values_chnuks   = torch.chunk(value, self.n_heads, -1)

        for head in range(0, self.n_heads):
            #print(self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_queries_chunks_{head}')])
            #print(self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_queries_chunks_{head}')]['Output'].shape)
            query_chnuks_matched    = tensors_within_tolerance(queries_chnuks[head],  self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_queries_chunks_{head}')], self.atol)
            key_chnuks_matched      = tensors_within_tolerance(keys_chnuks[head],     self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_keys_chunks_{head}')], self.atol)
            value_chnuks_matched    = tensors_within_tolerance(values_chnuks[head],   self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_values_chunks_{head}')], self.atol)
            print(f"query_chnuks_matched:   {query_chnuks_matched}")
            print(f"key_chnuks_matched:     {key_chnuks_matched}")
            print(f"value_chnuks_matched:   {value_chnuks_matched}")

        context_vecs = []
        for head in range(0, self.n_heads):
            #print(self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_context_vecs_{head}')])
            
            print(f"=================================== HEAD {head} ===================================\n")
            key_transposed = keys_chnuks[head].transpose(1, 2)
            key_transposed_matched = tensors_within_tolerance(key_transposed, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_keys_transposed_{head}')], self.atol)
            print(f"key_transposed_matched:   {key_transposed_matched}")

            attention_scores = queries_chnuks[head] @ key_transposed
            attention_scores_matched = tensors_within_tolerance(attention_scores, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_attention_scores_{head}')], self.atol)
            print(f"attention_scores_matched:   {attention_scores_matched}")


            attention_scores_scaled = attention_scores * 1/math.sqrt(keys_chnuks[head].shape[1])
            attention_scores_scaled_matched = tensors_within_tolerance(attention_scores_scaled, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_attention_scores_scaled_{head}')], self.atol)
            print(f"attention_scores_scaled_matched:   {attention_scores_scaled_matched}")

            attention_weights = F.softmax(attention_scores_scaled, dim=-1)
            attention_weights_matched = tensors_within_tolerance(attention_weights, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_attention_weights_{head}')], self.atol)
            print(f"attention_weights_matched:   {attention_weights_matched}")

            context_vec = attention_weights @ values_chnuks[head]
            context_vec_matched = tensors_within_tolerance(context_vec, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_context_vecs_{head}')], self.atol)
            print(f"context_vec_matched:   {context_vec_matched}")

            context_vecs.append(context_vec)
        #     print(f"=================================================================================\n")

        concat_heads = torch.cat(context_vecs, dim=-1)
        concat_heads_matched = tensors_within_tolerance(concat_heads, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_concat_heads')], self.atol)
        print(f"concat_heads_matched:   {concat_heads_matched}")

        projected_context_vecs = self.heads_proj(concat_heads)
        projected_context_vecs_matched = tensors_within_tolerance(projected_context_vecs, self.self_attention_layer_c[find_key(self.self_attention_layer_c, f'{self.layer_name}_heads_proj')]['Output'], self.atol)
        print(f"projected_context_vecs_matched:   {projected_context_vecs_matched}")



In [62]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, feed_forward_network_c):
        super().__init__()
        self.input = nn.Linear(2, 5)
        self.output = nn.Linear(5, 2)

    def forward(self, x):
        pass

In [63]:
class Transformer(nn.Module):
    def __init__(self, transformer_layer_c, n_heads, atol):
        super().__init__() 
        self.n_heads = n_heads
        self.atol = atol
        self_attention_layer_c = {k: v for k, v in transformer_layer_c.items() if "self_attention_layer" in k}
        feed_forward_network_c = {k: v for k, v in transformer_layer_c.items() if "feed_forward_network" in k}
        self.self_attention_multi_head  = SelfAttentionMultiHead(self_attention_layer_c, n_heads, atol)
        #self.feed_forward_network       = FeedForwardNetwork(feed_forward_network_c)
    
    def forward(self, x):
        x = self.self_attention_multi_head(x)
        #x = self.feed_forward_network(x)
        return x


In [64]:
class TransformerBlock(nn.Module):
    def __init__(self, transformer_block_c, n_layers, n_heads, atol):
        super().__init__()
        self.n_layers = n_layers
        self.atol = atol
        self.transformer_layers = {}
        for layer_no in range(0, n_layers):
            transformer_layer_c = {k: v for k, v in transformer_block_c.items() if k.startswith(f"transformer_layer_{layer_no}")}
            self.transformer_layers[layer_no] = Transformer(transformer_layer_c, n_heads, atol)

    def forward(self, x):
        for layer_no in range(0, self.n_layers):
            x = self.transformer_layers[layer_no](x)
        return x

In [65]:
class GPT(nn.Module):
    def __init__(self, gpt_model_c, n_layers, n_heads, atol):
        super().__init__()   
        self.atol = atol
           
        num_token_embeddings            = gpt_model_c['gpt_model.token_embed_layer']['Weights'].shape[0]
        token_embedding_dim             = gpt_model_c['gpt_model.token_embed_layer']['Weights'].shape[1]
        num_pos_embeddings              = gpt_model_c['gpt_model.pos_embed_layer']['Weights'].shape[0]
        pos_embedding_dim               = gpt_model_c['gpt_model.pos_embed_layer']['Weights'].shape[1]

        print(f"num_token_embeddings: {num_token_embeddings}")
        print(f"token_embedding_dim: {token_embedding_dim}")
        print(f"num_pos_embeddings: {num_pos_embeddings}")
        print(f"pos_embedding_dim: {pos_embedding_dim}")

        self.token_embeddings_layer     = nn.Embedding(num_embeddings=num_token_embeddings, embedding_dim=token_embedding_dim)
        self.pos_embeddings_layer       = nn.Embedding(num_embeddings=num_pos_embeddings, embedding_dim=pos_embedding_dim)

        self.token_embeddings_layer.weight.data.copy_(gpt_model_c['gpt_model.token_embed_layer']['Weights'])
        self.pos_embeddings_layer.weight.data.copy_(gpt_model_c['gpt_model.pos_embed_layer']['Weights'])
        
 
        assert(gpt_model_c['gpt_model.token_embed_layer']['Weights'].shape == self.token_embeddings_layer.weight.shape)
        assert(gpt_model_c['gpt_model.token_embed_layer']['Weights'].shape == self.pos_embeddings_layer.weight.shape)

        
        self.token_embeddings_c         = gpt_model_c['gpt_model.token_embed_layer']['Output']
        self.pos_embeddings_c           = gpt_model_c['gpt_model.pos_embed_layer']['Output']
        self.input_embeddings_c         = gpt_model_c['gpt_model.workspace.input_embeddings']
        self.position_indicies_c        = gpt_model_c['gpt_model.workspace.position_indices'].long()

        transformer_block_c = {k: v for k, v in gpt_model_c.items() if "transformer_layer" in k}
        self.transformer_block = TransformerBlock(transformer_block_c, n_layers, n_heads, self.atol)

    def forward(self, input_tokens_c):
        #print(f"x.shape:    {input_tokens_c.shape}")
        token_embeddings    = self.token_embeddings_layer(input_tokens_c)[0]
        print(self.position_indicies_c)
        pos_embeddings      = self.pos_embeddings_layer(self.position_indicies_c)[0]
        input_embeddings    = token_embeddings + pos_embeddings


        token_embeddings_matched    = tensors_within_tolerance(token_embeddings,    self.token_embeddings_c,    self.atol)
        pos_embeddings_matched      = tensors_within_tolerance(pos_embeddings,      self.pos_embeddings_c,      self.atol)
        input_embeddings_matched    = tensors_within_tolerance(input_embeddings,    self.input_embeddings_c,    self.atol)

        print(f"token_embeddings_matched:   {token_embeddings_matched}")
        print(f"pos_embeddings_matched:     {pos_embeddings_matched}")
        print(f"input_embeddings_matched:   {input_embeddings_matched}")


        # print("***********Token Embeddings***********\n")
        # print(self.token_embeddings_c[0], "\n")
        # print(token_embeddings[0], "\n\n")

        # print("***********Pose Embeddings***********\n")
        # print(self.pos_embeddings_c[0], "\n")
        # print(pos_embeddings[0], "\n\n")


        # print("***********Input Embeddings***********\n")
        # print(self.input_embeddings_c[0], "\n")
        # print(input_embeddings[0], "\n\n")

        # assert(
        #     token_embeddings_matched and 
        #     pos_embeddings_matched and 
        #     input_embeddings_matched
        # )
        
        contextual_embddings = self.transformer_block(input_embeddings)


        # print(token_embeddings - self.token_embed_layer_output_c)
        return contextual_embddings

In [67]:
atol = 0.00000001
gpt = GPT(gpt_model_c=gpt_model_c, n_layers=1, n_heads=2, atol=atol)
input_embeddings = gpt(gpt_model_c['input_tokens'].long())

num_token_embeddings: 6
token_embedding_dim: 32
num_pos_embeddings: 6
pos_embedding_dim: 32
tensor([[[0, 1, 2, 3, 4, 5],
         [0, 1, 2, 3, 4, 5]]])
max |diff|: 0.0
max |diff|: 0.0
max |diff|: 0.0
token_embeddings_matched:   True
pos_embeddings_matched:     True
input_embeddings_matched:   True
max |diff|: 0.0
max |diff|: 0.0
max |diff|: 0.0
query_matched:   True
key_matched:     True
value_matched:   True
max |diff|: 0.0
max |diff|: 0.0
max |diff|: 0.0
query_chnuks_matched:   True
key_chnuks_matched:     True
value_chnuks_matched:   True
max |diff|: 0.0
max |diff|: 0.0
max |diff|: 0.0
query_chnuks_matched:   True
key_chnuks_matched:     True
value_chnuks_matched:   True
=================================== HEAD 0 ===================================

max |diff|: 0.0
key_transposed_matched:   True
max |diff|: 0.0
attention_scores_matched:   True
max |diff|: 0.0
attention_scores_scaled_matched:   True
max |diff|: 0.0
attention_weights_matched:   True
max |diff|: 0.0
context_vec_matched

In [57]:
for key in gpt_model_c.keys():
    print(key)

transformer_layer_0__self_attention_layer_context_vecs_3
transformer_layer_0__self_attention_layer_attention_scores_0
gpt_model.pos_embed_layer
transformer_layer_0__self_attention_layer_attention_scores_1
transformer_layer_0__self_attention_layer_context_vecs_2
input_tokens
transformer_layer_0__self_attention_layer_context_vecs_0
transformer_layer_0__self_attention_layer_w_key
transformer_layer_0__self_attention_layer_attention_scores_3
transformer_layer_0__self_attention_layer_attention_scores_2
transformer_layer_0__self_attention_layer_context_vecs_1
transformer_layer_0__self_attention_layer_context_vecs_5
transformer_layer_0__self_attention_layer_values_chunks_9
transformer_layer_0__self_attention_layer_attention_scores_6
transformer_layer_0__self_attention_layer_attention_scores_7
transformer_layer_0__self_attention_layer_values_chunks_8
transformer_layer_0__self_attention_layer_context_vecs_4
transformer_layer_0__self_attention_layer_context_vecs_6
transformer_layer_0__self_attent

In [595]:
key_transposed_global.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [333]:
# class SelfAttention:
#     def __init__(self, self_attention_layer_c):
#         W_query_weights =   self_attention_layer_c['W_Query']
#         W_key_weights =     self_attention_layer_c['W_Key']
#         W_value_weights =   self_attention_layer_c['W_Value']

#         W_query_weights = W_query_weights.t()
#         W_key_weights = W_key_weights.t()
#         W_value_weights = W_value_weights.t()

#         self.W_query = nn.Linear(W_query_weights.shape[0], W_query_weights.shape[1], bias=False)
#         self.W_query.weight = nn.Parameter(W_query_weights)

#         self.W_key = nn.Linear(W_key_weights.shape[0], W_key_weights.shape[1], bias=False)
#         self.W_key.weight = nn.Parameter(W_key_weights)

#         self.W_value = nn.Linear(W_value_weights.shape[0], W_value_weights.shape[1], bias=False)
#         self.W_value.weight = nn.Parameter(W_value_weights)
    
#     def forward(self, x):
#         print(f"Input Embeddings: \nShape: {x.shape}\n{x}")
#         print(f"W_Query\nShape: {self.W_query.weight.shape}\n{self.W_query.weight}")
#         print(f"W_Key\nShape:   {self.W_key.weight.shape}\n{self.W_key.weight}")
#         print(f"W_Value\nShape: {self.W_value.weight.shape}\n{self.W_value.weight}")
#         print("===========================================")

#         query = self.W_query(x)
#         key = self.W_key(x)
#         value = self.W_value(x)
        

#         # print(f"Query\nShape: {query.shape}\n{query}")
#         # print(f"Key\nShape:   {key.shape}\n{key}")
#         # print(f"Value\nShape: {value.shape}\n{value}")
#         # print("===========================================")

#         key_transposed = key.t()
#         print(f"Key transposed\nShape: {key_transposed.shape}\n{key_transposed}")

#         attention_scores = query @ key_transposed
#         print(f"Attention Scores\nShape: {attention_scores.shape}\n{attention_scores}")

#         attention_scores_scaled = attention_scores * 1/math.sqrt(key.shape[1])
#         print(f"Attention Scores Scaled\nShape: {attention_scores_scaled.shape}\n{attention_scores_scaled}")

#         attention_weights = F.softmax(attention_scores_scaled, dim=1)
#         print(f"Attention Weights\nShape: {attention_weights.shape}\n{attention_weights}")

#         context_vecs = attention_weights @ value
#         print(f"Context Vecs\nShape: {context_vecs.shape}\n{context_vecs}")





In [332]:
# self_attention = SelfAttention(self_attention_layer_c)

# self_attention.forward(input_embeddings_c)

## Safe Tensors

In [547]:
from safetensors import safe_open

path = "/Users/uonliaquat/Downloads/model.safetensors"

with safe_open(path, framework="pt", device="cpu") as f:
    keys = list(f.keys())

for key in keys:
    print(key)

h.0.attn.bias
h.0.attn.c_attn.bias
h.0.attn.c_attn.weight
h.0.attn.c_proj.bias
h.0.attn.c_proj.weight
h.0.ln_1.bias
h.0.ln_1.weight
h.0.ln_2.bias
h.0.ln_2.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_proj.bias
h.0.mlp.c_proj.weight
h.1.attn.bias
h.1.attn.c_attn.bias
h.1.attn.c_attn.weight
h.1.attn.c_proj.bias
h.1.attn.c_proj.weight
h.1.ln_1.bias
h.1.ln_1.weight
h.1.ln_2.bias
h.1.ln_2.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_proj.bias
h.1.mlp.c_proj.weight
h.10.attn.bias
h.10.attn.c_attn.bias
h.10.attn.c_attn.weight
h.10.attn.c_proj.bias
h.10.attn.c_proj.weight
h.10.ln_1.bias
h.10.ln_1.weight
h.10.ln_2.bias
h.10.ln_2.weight
h.10.mlp.c_fc.bias
h.10.mlp.c_fc.weight
h.10.mlp.c_proj.bias
h.10.mlp.c_proj.weight
h.11.attn.bias
h.11.attn.c_attn.bias
h.11.attn.c_attn.weight
h.11.attn.c_proj.bias
h.11.attn.c_proj.weight
h.11.ln_1.bias
h.11.ln_1.weight
h.11.ln_2.bias
h.11.ln_2.weight
h.11.mlp.c_fc.bias
h.11.mlp.c_fc.weight
h.11.mlp.c_proj.bias
h.11.mlp.c_proj.weight
h.2.at

In [546]:
import torch
from transformers import GPT2Model

# Load GPT-2 small
model = GPT2Model.from_pretrained("gpt2")
model.eval()

# Dictionary to store all activations
activations = {}

# Generic hook for all modules
def register_all_hooks(model):
    """
    Registers a forward hook for every submodule in the model.
    Stores the output of each module in the activations dict.
    """
    def make_hook(name):
        def hook(module, inputs, output):
            # Only store tensors (or tuples of tensors)
            if torch.is_tensor(output):
                activations[name] = output.detach().cpu()
            elif isinstance(output, (tuple, list)):
                # handle modules that return tuples
                activations[name] = tuple(
                    o.detach().cpu() if torch.is_tensor(o) else o
                    for o in output
                )
        return hook

    for name, module in model.named_modules():
        module.register_forward_hook(make_hook(name))

# Register hooks on all modules
register_all_hooks(model)

# Example input
input_ids = torch.tensor([[464, 318, 257, 1332]])  # "Once upon a time"

# Forward pass
with torch.no_grad():
    model(input_ids)

# Check all stored activations
for k in activations:
    print(f"{k}: {activations[k].shape if torch.is_tensor(activations[k]) else 'tuple'}")

wte: torch.Size([1, 4, 768])
wpe: torch.Size([1, 4, 768])
drop: torch.Size([1, 4, 768])
h.0.ln_1: torch.Size([1, 4, 768])
h.0.attn.c_attn: torch.Size([1, 4, 2304])
h.0.attn.c_proj: torch.Size([1, 4, 768])
h.0.attn.resid_dropout: torch.Size([1, 4, 768])
h.0.attn: tuple
h.0.ln_2: torch.Size([1, 4, 768])
h.0.mlp.c_fc: torch.Size([1, 4, 3072])
h.0.mlp.act: torch.Size([1, 4, 3072])
h.0.mlp.c_proj: torch.Size([1, 4, 768])
h.0.mlp.dropout: torch.Size([1, 4, 768])
h.0.mlp: torch.Size([1, 4, 768])
h.0: tuple
h.1.ln_1: torch.Size([1, 4, 768])
h.1.attn.c_attn: torch.Size([1, 4, 2304])
h.1.attn.c_proj: torch.Size([1, 4, 768])
h.1.attn.resid_dropout: torch.Size([1, 4, 768])
h.1.attn: tuple
h.1.ln_2: torch.Size([1, 4, 768])
h.1.mlp.c_fc: torch.Size([1, 4, 3072])
h.1.mlp.act: torch.Size([1, 4, 3072])
h.1.mlp.c_proj: torch.Size([1, 4, 768])
h.1.mlp.dropout: torch.Size([1, 4, 768])
h.1.mlp: torch.Size([1, 4, 768])
h.1: tuple
h.2.ln_1: torch.Size([1, 4, 768])
h.2.attn.c_attn: torch.Size([1, 4, 2304])
h.

In [563]:
import torch
from transformers import GPT2Model

model = GPT2Model.from_pretrained("gpt2")
model.eval()

activations = {}

def register_all_hooks(model):
    def make_hook(name):
        def hook(module, inputs, output):
            if torch.is_tensor(output):
                activations[name] = output.detach().cpu()
            elif isinstance(output, (tuple, list)):
                # handle modules that return tuples (e.g., attention)
                activations[name] = tuple(
                    o.detach().cpu() if torch.is_tensor(o) else o
                    for o in output
                )
        return hook

    for name, module in model.named_modules():
        module.register_forward_hook(make_hook(name))

register_all_hooks(model)

# Example run
input_ids = torch.tensor([[464, 318, 257, 1332]])  # "Once upon a time"
with torch.no_grad():
    model(input_ids)

In [564]:
for key in activations.keys():
    print(key)

wte
wpe
drop
h.0.ln_1
h.0.attn.c_attn
h.0.attn.c_proj
h.0.attn.resid_dropout
h.0.attn
h.0.ln_2
h.0.mlp.c_fc
h.0.mlp.act
h.0.mlp.c_proj
h.0.mlp.dropout
h.0.mlp
h.0
h.1.ln_1
h.1.attn.c_attn
h.1.attn.c_proj
h.1.attn.resid_dropout
h.1.attn
h.1.ln_2
h.1.mlp.c_fc
h.1.mlp.act
h.1.mlp.c_proj
h.1.mlp.dropout
h.1.mlp
h.1
h.2.ln_1
h.2.attn.c_attn
h.2.attn.c_proj
h.2.attn.resid_dropout
h.2.attn
h.2.ln_2
h.2.mlp.c_fc
h.2.mlp.act
h.2.mlp.c_proj
h.2.mlp.dropout
h.2.mlp
h.2
h.3.ln_1
h.3.attn.c_attn
h.3.attn.c_proj
h.3.attn.resid_dropout
h.3.attn
h.3.ln_2
h.3.mlp.c_fc
h.3.mlp.act
h.3.mlp.c_proj
h.3.mlp.dropout
h.3.mlp
h.3
h.4.ln_1
h.4.attn.c_attn
h.4.attn.c_proj
h.4.attn.resid_dropout
h.4.attn
h.4.ln_2
h.4.mlp.c_fc
h.4.mlp.act
h.4.mlp.c_proj
h.4.mlp.dropout
h.4.mlp
h.4
h.5.ln_1
h.5.attn.c_attn
h.5.attn.c_proj
h.5.attn.resid_dropout
h.5.attn
h.5.ln_2
h.5.mlp.c_fc
h.5.mlp.act
h.5.mlp.c_proj
h.5.mlp.dropout
h.5.mlp
h.5
h.6.ln_1
h.6.attn.c_attn
h.6.attn.c_proj
h.6.attn.resid_dropout
h.6.attn
h.6.ln_2
h.6.m

In [565]:
activations["h.0.ln_1"].shape

torch.Size([1, 4, 768])

In [737]:
from safetensors import safe_open

path = "/Users/uonliaquat/Downloads/model.safetensors"

with safe_open(path, framework="pt", device="cpu") as f:
    h0_keys = [k for k in f.keys() if k.startswith("h.0")]

for key in h0_keys:
    print(key)

h.0.attn.bias
h.0.attn.c_attn.bias
h.0.attn.c_attn.weight
h.0.attn.c_proj.bias
h.0.attn.c_proj.weight
h.0.ln_1.bias
h.0.ln_1.weight
h.0.ln_2.bias
h.0.ln_2.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_proj.bias
h.0.mlp.c_proj.weight


In [1276]:
path = "/Users/uonliaquat/Downloads/model.safetensors"

with safe_open(path, framework="pt", device="cpu") as f:
    # ln_weight = f.get_tensor("h.0.ln_1.weight")
    # ln_bias = f.get_tensor("h.0.ln_1.bias")
    h0_ln1_weight = f.get_tensor("h.0.ln_1.weight")
    h0_ln1_bias = f.get_tensor("h.0.ln_1.bias")


h0_ln1_weight.shape, h0_ln1_bias.shape,

(torch.Size([768]), torch.Size([768]))

## Read C Model

In [587]:
from safetensors import safe_open

filename = "/Users/uonliaquat/workspace/zerograd/c_model.safetensors"

output_c = {}

with safe_open(filename, framework="pt", device="cpu") as f:
    for key in f.keys():
        output_c[key] = f.get_tensor(key)

# Inspect
print(len(output_c))
print(list(output_c.keys()))

1512
['h.0.attn.0.attn_score', 'h.0.attn.0.attn_score_scaled', 'h.0.attn.0.attn_weight', 'h.0.attn.0.ctx_vec', 'h.0.attn.0.k_head', 'h.0.attn.0.key_transposed', 'h.0.attn.0.q_head', 'h.0.attn.0.v_head', 'h.0.attn.1.attn_score', 'h.0.attn.1.attn_score_scaled', 'h.0.attn.1.attn_weight', 'h.0.attn.1.ctx_vec', 'h.0.attn.1.k_head', 'h.0.attn.1.key_transposed', 'h.0.attn.1.q_head', 'h.0.attn.1.v_head', 'h.0.attn.10.attn_score', 'h.0.attn.10.attn_score_scaled', 'h.0.attn.10.attn_weight', 'h.0.attn.10.ctx_vec', 'h.0.attn.10.k_head', 'h.0.attn.10.key_transposed', 'h.0.attn.10.q_head', 'h.0.attn.10.v_head', 'h.0.attn.11.attn_score', 'h.0.attn.11.attn_score_scaled', 'h.0.attn.11.attn_weight', 'h.0.attn.11.ctx_vec', 'h.0.attn.11.k_head', 'h.0.attn.11.key_transposed', 'h.0.attn.11.q_head', 'h.0.attn.11.v_head', 'h.0.attn.2.attn_score', 'h.0.attn.2.attn_score_scaled', 'h.0.attn.2.attn_weight', 'h.0.attn.2.ctx_vec', 'h.0.attn.2.k_head', 'h.0.attn.2.key_transposed', 'h.0.attn.2.q_head', 'h.0.attn.2.v_

In [ ]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, weight, bias, eps=1e-5):
        super().__init__()
        self.normalized_shape = normalized_shape
        self.eps = eps

        # MUST be Parameters
        self.weight = nn.Parameter(weight.clone())
        self.bias = nn.Parameter(bias.clone())

    def forward(self, x):
        # Compute mean & variance over last dim
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        x_hat = (x - mean) / torch.sqrt(var + self.eps)

        # Explicit reshape (matches PyTorch internals)
        return x_hat * self.weight.view(1, 1, -1) + self.bias.view(1, 1, -1)
    

ln_ref = torch.nn.LayerNorm(768)
ln_custom = LayerNorm(
    768,
    ln_ref.weight.data,
    ln_ref.bias.data,
    ln_ref.eps
)

x = torch.randn(2, 5, 768)

print(torch.allclose(ln_ref(x), ln_custom(x), atol=1e-6))

True


In [311]:
ln_ref(x)

tensor([[[-0.3544,  0.3964,  0.2084,  ...,  0.0833,  0.1578,  1.7399],
         [ 0.6083,  1.5527,  1.6953,  ..., -0.0332, -1.1895,  0.5943],
         [ 0.4841, -0.3757, -1.1099,  ..., -2.5548,  0.6279,  0.8806],
         [ 0.3732,  0.2516, -1.4893,  ...,  1.1755,  0.0857,  0.6072],
         [ 0.9265,  1.1821, -1.2627,  ..., -0.8215, -0.0867,  0.4206]],

        [[ 0.2936, -0.1163,  2.9201,  ..., -0.8842, -0.5789, -0.0855],
         [ 1.7568, -1.1778, -0.5492,  ...,  0.8829,  1.2080, -0.4937],
         [ 0.4102, -0.0173, -0.9737,  ...,  0.4693,  0.6395, -0.6874],
         [ 0.2136, -0.4779,  0.1569,  ...,  0.8548,  0.1354, -0.6189],
         [-0.5753,  0.5244,  0.9420,  ...,  1.0487,  0.5651, -0.0618]]],
       grad_fn=<NativeLayerNormBackward0>)

## Genrating Vocab

In [624]:
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokens = tokenizer.convert_ids_to_tokens(list(range(tokenizer.vocab_size)))

# Choose a separator (must not appear in any token)
SEP = "|"  # safe ASCII separator

def clean_token(tok):
    tok = tok.replace('Ġ', ' ')   # space marker
    tok = tok.replace('Ċ', '<NL>')  # newline marker as visible string
    # Optional: replace any other non-ASCII sequences
    tok = ''.join(c if ord(c) < 128 else '<U>' for c in tok)
    return tok

with open("gpt2_vocab.txt", "w", encoding="utf-8") as f:
    f.write(SEP.join(clean_token(tok) for tok in tokens))

## Python GPT

In [590]:
import math
import torch
import torch.nn as nn
from transformers import GPT2Model

import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import torch
from torch import nn

import torch
from torch import nn

class LayerNorm(nn.Module):
    def __init__(
        self,
        normalized_shape,
        eps: float = 1e-5,
        elementwise_affine: bool = True,
        device=None,
        dtype=None,
    ):
        super().__init__()

        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)
        self.normalized_shape = tuple(normalized_shape)
        self.eps = eps
        self.elementwise_affine = elementwise_affine

        factory_kwargs = {"device": device, "dtype": dtype}

        if elementwise_affine:
            self.weight = nn.Parameter(
                torch.ones(self.normalized_shape, **factory_kwargs)
            )
            self.bias = nn.Parameter(
                torch.zeros(self.normalized_shape, **factory_kwargs)
            )
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x, out, h, l) -> torch.Tensor:
        # Normalize over the last len(normalized_shape) dimensions
        dims = tuple(range(-len(self.normalized_shape), 0))

        mean = x.mean(dim=dims, keepdim=True)
        #out[f'h.{h}.ln_1.mean'] = mean
        var = x.var(dim=dims, keepdim=True, unbiased=False)
        #out[f'h.{h}.ln_1.var'] = var
        out[f'h.{h}.ln_{l}.mean_var'] = torch.cat([mean, var], dim=-1)
        x_hat = (x - mean) / torch.sqrt(var + self.eps)

        out[f'h.{h}.ln_{l}.x_norm'] = x_hat

        if self.elementwise_affine:
            x_hat = x_hat * self.weight
            out[f'h.{h}.ln_{l}.x_norm_scaled'] = x_hat
            x_hat = x_hat + self.bias
            out[f'h.{h}.ln_{l}.x_norm_shifted'] = x_hat
        return x_hat

    def extra_repr(self) -> str:
        return (
            f"normalized_shape={self.normalized_shape}, "
            f"eps={self.eps}, "
            f"elementwise_affine={self.elementwise_affine}"
        )
    

class GPT2_Full_Debug(nn.Module):
    def __init__(self, model_name="gpt2", device="cpu", dtype=torch.float32):
        super().__init__()

        gpt2 = GPT2Model.from_pretrained(model_name)
        gpt2.eval()

        self.n_heads = gpt2.config.n_head
        self.hidden_size = gpt2.config.n_embd
        self.head_dim = self.hidden_size // self.n_heads
        self.n_layers = gpt2.config.n_layer
        self.vocab_size = gpt2.config.vocab_size

        # Embeddings
        self.wte = nn.Embedding.from_pretrained(gpt2.wte.weight.detach().to(dtype), freeze=True)
        self.wpe = nn.Embedding.from_pretrained(gpt2.wpe.weight.detach().to(dtype), freeze=True)

        # Transformer blocks
        self.blocks = nn.ModuleList()
        for i, block in enumerate(gpt2.h):
            b = nn.ModuleDict({
                "ln_1": LayerNorm(self.hidden_size, eps=block.ln_1.eps),
                "c_attn": nn.Linear(self.hidden_size, 3 * self.hidden_size),
                "c_proj_attn": nn.Linear(self.hidden_size, self.hidden_size),
                "ln_2": LayerNorm(self.hidden_size, eps=block.ln_2.eps),
                "c_fc": nn.Linear(self.hidden_size, 4 * self.hidden_size),
                "c_proj_mlp": nn.Linear(4 * self.hidden_size, self.hidden_size),
            })
            # Copy weights
            b["ln_1"].weight.data.copy_(block.ln_1.weight)
            b["ln_1"].bias.data.copy_(block.ln_1.bias)

            b["c_attn"].weight.data.copy_(block.attn.c_attn.weight.T)
            b["c_attn"].bias.data.copy_(block.attn.c_attn.bias)

            b["c_proj_attn"].weight.data.copy_(block.attn.c_proj.weight.T)
            b["c_proj_attn"].bias.data.copy_(block.attn.c_proj.bias)

            b["ln_2"].weight.data.copy_(block.ln_2.weight)
            b["ln_2"].bias.data.copy_(block.ln_2.bias)

            b["c_fc"].weight.data.copy_(block.mlp.c_fc.weight.T)
            b["c_fc"].bias.data.copy_(block.mlp.c_fc.bias)

            b["c_proj_mlp"].weight.data.copy_(block.mlp.c_proj.weight.T)
            b["c_proj_mlp"].bias.data.copy_(block.mlp.c_proj.bias)

            self.blocks.append(b)

        # Final layer norm
        self.ln_f = nn.LayerNorm(self.hidden_size, eps=gpt2.ln_f.eps)
        self.ln_f.weight.data.copy_(gpt2.ln_f.weight)
        self.ln_f.bias.data.copy_(gpt2.ln_f.bias)

        # LM head (weight tied)
        self.lm_head = nn.Linear(self.hidden_size, self.vocab_size, bias=False)
        self.lm_head.weight.data.copy_(self.wte.weight)

        self.to(device=device, dtype=dtype)

    def forward(self, input_ids):
        bsz, seq_len = input_ids.shape
        device = input_ids.device
        out = {}

        # Embeddings
        pos_ids = torch.arange(seq_len, device=device).unsqueeze(0)
        tok_emb = self.wte(input_ids)
        pos_emb = self.wpe(pos_ids)
        x = tok_emb + pos_emb
        out["token_emb"] = tok_emb
        out["pos_emb"] = pos_emb

        # Iterate over blocks
        for layer_idx, b in enumerate(self.blocks):
            #layer_out = {}
            out[f"h.{layer_idx}.input_embedding"] = x

            # LN1
            x_ln1 = b["ln_1"](x, out, layer_idx, 1)
            out[f"h.{layer_idx}.ln_1"] = x_ln1

            # QKV
            qkv = b["c_attn"](x_ln1)
            q, k, v = qkv.split(self.hidden_size, dim=2)
            out[f"h.{layer_idx}.qkv"] = qkv
            out[f"h.{layer_idx}.attn.q"] = q
            out[f"h.{layer_idx}.attn.k"] = k
            out[f"h.{layer_idx}.attn.v"] = v

            # Split heads
            def split_heads(x):
                return x.view(bsz, seq_len, self.n_heads, self.head_dim).transpose(1, 2)

            qh = split_heads(q)[0]
            kh = split_heads(k)[0]
            vh = split_heads(v)[0]

            qh_list = torch.split(qh, 1, dim=0)  # split into chunks of size 1
            kh_list = torch.split(kh, 1, dim=0)  # split into chunks of size 1
            vh_list = torch.split(vh, 1, dim=0)  # split into chunks of size 1 
            
            for i, (q, k, v) in enumerate(zip(qh_list, kh_list, vh_list)):

                out[f"h.{layer_idx}.attn.{i}.q_head"] = q
                out[f"h.{layer_idx}.attn.{i}.k_head"] = k
                out[f"h.{layer_idx}.attn.{i}.v_head"] = v

            # Attention
            kh_t = kh.transpose(-2, -1)
            qk = torch.matmul(qh, kh_t)
            qk_scaled = qk / math.sqrt(self.head_dim)
            causal_mask = torch.tril(torch.ones(seq_len, seq_len, device=device)).view(1, 1, seq_len, seq_len)
            qk_masked = qk_scaled.masked_fill(causal_mask == 0, float("-inf"))
            attn_probs = torch.softmax(qk_masked, dim=-1)
            attn_ctx_heads = torch.matmul(attn_probs, vh)
            attn_ctx = attn_ctx_heads.transpose(1, 2).contiguous().view(bsz, seq_len, self.hidden_size)
            attn_out = b["c_proj_attn"](attn_ctx)

            out[f"h.{layer_idx}.attn.c_proj"] = attn_out

            #resid 1
            x = x + attn_out
            out[f"h.{layer_idx}.resid.1"] = x

            # LN2 + MLP
            x_ln2 = b["ln_2"](x, out, layer_idx, 2)
            mlp_out = b["c_proj_mlp"](torch.nn.functional.gelu(b["c_fc"](x_ln2)))
            out[f"h.{layer_idx}.mlp"] = mlp_out

            # resid 2
            x = x + mlp_out
            out[f"h.{layer_idx}.resid.2"] = x

            # out[f"h.{layer_idx}.ln_2"] = x_ln2
            # out[f"h.{layer_idx}.mlp_fc_out"] = mlp_out
            # out[f"h.{layer_idx}.block_output"] = x

            #out[f"layer_{layer_idx}"] = layer_out

        # Final layer norm
        x_final_ln = self.ln_f(x)
        out["ln_f"] = x_final_ln

        # LM head
        logits = self.lm_head(x_final_ln)
        out["head"] = logits
        probs = torch.softmax(logits, dim=-1)
        # out["logits"] = logits
        out["probs"] = probs

        # Greedy next token
        next_token_prob_dist = probs[:, -1, :]
        out['next_token_prob_dist'] = next_token_prob_dist
        next_token_id = torch.argmax(next_token_prob_dist, dim=-1)
        out["next_token_id"] = next_token_id

        return out

In [646]:
import torch
from transformers import GPT2Tokenizer

# -------------------------------
# Device
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------------
# Paragraph
# -------------------------------
#paragraph = ("Hello I'm a language model")

paragraph = (
    "In recent years, artificial intelligence has become an important tool "
    "in many industries, helping researchers analyze data, automate tasks, "
    "and improve decision making. Machine learning models are trained on "
    "large datasets and can generate text, recognize patterns, and predict "
    "outcomes based on previous examples."
)



# -------------------------------
# Tokenizer
# -------------------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input_ids = tokenizer.encode(paragraph, return_tensors="pt").to(device)
# print("token_ids: ", input_ids)


# -------------------------------
# Set context window to 6
# -------------------------------
context_len = 52
if input_ids.shape[1] > context_len:
    input_ids = input_ids[:, -context_len:]  # keep last 52 tokens

cur_len = input_ids.shape[1]

# -------------------------------
# Initialize GPT2 debug model
# -------------------------------
model = GPT2_Full_Debug(model_name="gpt2", device=device, dtype=torch.float32)
model.eval()

# -------------------------------
# Prepare for generation
# -------------------------------
num_generate = 200  # number of new tokens
generated_ids = input_ids[0].tolist()  # start with initial context

# Store intermediate outputs for all steps
all_steps_outputs = []

# -------------------------------
# Generation loop
# -------------------------------
for step in range(num_generate):
    cur_input = torch.tensor([generated_ids[-context_len:]], device=device)
    #print(cur_input)
    with torch.no_grad():
        outputs = model(cur_input)

    # Save intermediate outputs for this step
    all_steps_outputs.append(outputs)

    # Get next token (greedy)
    next_token_id = outputs['next_token_id'].item()

    # Append to generated list
    generated_ids.append(next_token_id)

# -------------------------------
# Decode full generated sequence
# -------------------------------
full_text = tokenizer.decode(generated_ids)
#print("\n=== Full generated text (initial + 50 new tokens) ===\n")
print(full_text)

# -------------------------------
# Optional: print next token for first 5 steps
# -------------------------------
# print("\n=== First 5 generated token IDs and strings ===")
# for step_out in all_steps_outputs:
#     tid = step_out['next_token_id'].item()
#     print(tid, tokenizer.decode([tid]))

output_p = all_steps_outputs[0]
# list(output_p.keys())

In recent years, artificial intelligence has become an important tool in many industries, helping researchers analyze data, automate tasks, and improve decision making. Machine learning models are trained on large datasets and can generate text, recognize patterns, and predict outcomes based on previous examples.

The goal of the study was to determine whether machine learning models can be used to predict the future of a large dataset. The goal of the study was to determine whether machine learning models can be used to predict the future of a large dataset.

The study was funded by the National Science Foundation (NSF) and the National Science Foundation (NSF) by the National Science Foundation (NSF) and the National Science Foundation (NSF) by the National Science Foundation (NSF) and the National Science Foundation (NSF) by the National Science Foundation (NSF) and the National Science Foundation (NSF) by the National Science Foundation (NSF) and the National Science Foundation (NS

In [623]:
tokenizer.decode([198])

'\n'

In [592]:
atol = 1e-4
h = 0
for h in range(0, 12):
    print(torch.allclose(output_p['token_emb'],  output_c['token_emb'], atol=atol))
    print(torch.allclose(output_p['pos_emb'],  output_c['pos_emb'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.input_embedding'], output_c[f'h.{h}.input_embedding'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_1.mean_var'], output_c[f'h.{h}.ln_1.mean_var'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_1.x_norm'], output_c[f'h.{h}.ln_1.x_norm'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_1.x_norm_shifted'], output_c[f'h.{h}.ln_1.x_norm_shifted'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_1.x_norm_scaled'], output_c[f'h.{h}.ln_1.x_norm_scaled'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_1'], output_c[f'h.{h}.ln_1'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.attn.q'], output_c[f'h.{h}.attn.q'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.attn.k'], output_c[f'h.{h}.attn.k'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.attn.v'], output_c[f'h.{h}.attn.v'], atol=atol))


    for i in range(0, 12):
        print(torch.allclose(output_p[f"h.{h}.attn.{i}.q_head"], output_c[f"h.{h}.attn.{i}.q_head"], atol=atol))
        print(torch.allclose(output_p[f"h.{h}.attn.{i}.k_head"], output_c[f"h.{h}.attn.{i}.k_head"], atol=atol))
        print(torch.allclose(output_p[f"h.{h}.attn.{i}.v_head"], output_c[f"h.{h}.attn.{i}.v_head"], atol=atol))


    #c_proj
    print(torch.allclose(output_p[f"h.{0}.attn.c_proj"], output_c[f"h.{0}.attn.c_proj"], atol=atol))

    #residual connection 1
    print(torch.allclose(output_p[f"h.{0}.resid.1"], output_c[f"h.{0}.resid.1"], atol=atol))

    # ln_2
    print(torch.allclose(output_p[f'h.{h}.ln_2.mean_var'], output_c[f'h.{h}.ln_2.mean_var'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_2.x_norm'], output_c[f'h.{h}.ln_2.x_norm'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_2.x_norm_shifted'], output_c[f'h.{h}.ln_2.x_norm_shifted'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.ln_2.x_norm_scaled'], output_c[f'h.{h}.ln_2.x_norm_scaled'], atol=atol))
    print(torch.allclose(output_p[f'h.{h}.mlp'], output_c[f'h.{h}.mlp'], atol=atol))

    #residual connection 2
    print(torch.allclose(output_p[f"h.{0}.resid.2"], output_c[f"h.{0}.resid.2"], atol=atol))


print(torch.allclose(output_p[f"ln_f"], output_c[f"ln_f"], atol=atol))
print(torch.allclose(output_p[f"head"], output_c[f"head"], atol=atol))
print(torch.allclose(output_p[f"probs"], output_c[f"probs"], atol=atol))
print(torch.allclose(output_p[f"next_token_prob_dist"], output_c[f"next_token_prob_dist"], atol=atol))



True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [593]:
print(output_p['next_token_id'])

tensor([263])


In [589]:
output_p[f"probs"].shape

torch.Size([1, 6, 50257])

In [566]:
output_p[f'h.{0}.mlp']

tensor([[[ 0.5170, -0.7297,  1.0580,  ..., -0.9128,  0.1060, -1.7272],
         [-0.6918, -0.1707,  0.4360,  ..., -0.7993,  0.4888,  0.2977],
         [-0.1759, -1.1365, -1.0272,  ..., -0.3013, -0.3815,  0.3796],
         [-0.6328,  0.5997,  0.4721,  ...,  0.0642, -0.2357, -0.1439],
         [-1.5716,  0.3739, -0.8954,  ...,  0.1883,  0.7078,  1.2277],
         [-1.6475, -0.7412, -0.5769,  ..., -0.8543,  0.3709,  0.8488]]])

In [440]:
tensors_list = torch.split(output_p[f"h.{h}.attn.q_heads"][0], 1, dim=0)
tensors_list[0].shape

torch.Size([1, 6, 64])

In [532]:
list(output_c.keys())

['h.0.attn.0.attn_score',
 'h.0.attn.0.attn_score_scaled',
 'h.0.attn.0.attn_weight',
 'h.0.attn.0.ctx_vec',
 'h.0.attn.0.k_head',
 'h.0.attn.0.key_transposed',
 'h.0.attn.0.q_head',
 'h.0.attn.0.v_head',
 'h.0.attn.1.attn_score',
 'h.0.attn.1.attn_score_scaled',
 'h.0.attn.1.attn_weight',
 'h.0.attn.1.ctx_vec',
 'h.0.attn.1.k_head',
 'h.0.attn.1.key_transposed',
 'h.0.attn.1.q_head',
 'h.0.attn.1.v_head',
 'h.0.attn.10.attn_score',
 'h.0.attn.10.attn_score_scaled',
 'h.0.attn.10.attn_weight',
 'h.0.attn.10.ctx_vec',
 'h.0.attn.10.k_head',
 'h.0.attn.10.key_transposed',
 'h.0.attn.10.q_head',
 'h.0.attn.10.v_head',
 'h.0.attn.11.attn_score',
 'h.0.attn.11.attn_score_scaled',
 'h.0.attn.11.attn_weight',
 'h.0.attn.11.ctx_vec',
 'h.0.attn.11.k_head',
 'h.0.attn.11.key_transposed',
 'h.0.attn.11.q_head',
 'h.0.attn.11.v_head',
 'h.0.attn.2.attn_score',
 'h.0.attn.2.attn_score_scaled',
 'h.0.attn.2.attn_weight',
 'h.0.attn.2.ctx_vec',
 'h.0.attn.2.k_head',
 'h.0.attn.2.key_transposed',
 'h

In [350]:
import torch
from transformers import GPT2Model
torch.set_printoptions(sci_mode=False)

# Load GPT-2 small (124M)
model = GPT2Model.from_pretrained("gpt2")
model.eval()

# First transformer block
block0 = model.h[0]

# First LayerNorm in the block
ln1 = block0.ln_1

# Print shapes
print("ln_1 weight shape:", ln1.weight.shape)
print("ln_1 bias shape:  ", ln1.bias.shape)

# # Print values
print("\nln_1 weight:")
print(ln1.weight)

# print("\nln_1 bias:")
# print(ln1.bias)

ln_1 weight shape: torch.Size([768])
ln_1 bias shape:   torch.Size([768])

ln_1 weight:
Parameter containing:
tensor([0.2232, 0.1820, 0.1534, 0.1917, 0.2036, 0.1948, 0.1467, 0.1865, 0.2143,
        0.1956, 0.2118, 0.2153, 0.1882, 0.2074, 0.1871, 0.2040, 0.2044, 0.1900,
        0.1952, 0.0475, 0.1909, 0.2115, 0.1971, 0.2202, 0.1998, 0.2108, 0.2303,
        0.1879, 0.1939, 0.2018, 0.1891, 0.1861, 0.1958, 0.1832, 0.1978, 0.2243,
        0.0706, 0.1958, 0.1943, 0.1939, 0.1978, 0.1951, 0.1995, 0.1912, 0.2083,
        0.2037, 0.1849, 0.1945, 0.2189, 0.0419, 0.1977, 0.1979, 0.0608, 0.1824,
        0.2055, 0.0476, 0.1892, 0.2079, 0.2047, 0.2233, 0.2097, 0.2075, 0.2076,
        0.1793, 0.1312, 0.1841, 0.1939, 0.1561, 0.0577, 0.1948, 0.2048, 0.1717,
        0.1942, 0.1708, 0.1989, 0.1993, 0.2082, 0.1071, 0.1968, 0.1770, 0.2164,
        0.1864, 0.1938, 0.2184, 0.1343, 0.1707, 0.0683, 0.1401, 0.1823, 0.2045,
        0.2007, 0.1853, 0.1783, 0.1889, 0.1870, 0.1975, 0.2114, 0.2108, 0.2083,
        0.

In [634]:
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def token_to_text(token_id):
    # Convert token ID to its token string
    return tokenizer.convert_ids_to_tokens(token_id)

# Example
token_id = 1127

token_text = token_to_text(token_id)
print(f"Token ID {token_id}: {token_text}")

Token ID 1127: ines


In [644]:
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# File to store vocab in fixed 64-byte slots
with open("gpt2_vocab.bin", "wb") as f:
    for token_id in range(tokenizer.vocab_size):
        # Get decoded token
        decoded_text = tokenizer.decode([token_id])
        
        # Encode as UTF-8 bytes
        token_bytes = decoded_text.encode('utf-8')
        
        # Truncate if longer than 64 bytes
        if len(token_bytes) > 64:
            token_bytes = token_bytes[:64]
        
        # Pad with zeros to make it exactly 64 bytes
        token_bytes = token_bytes.ljust(64, b'\x00')
        
        # Write to file
        f.write(token_bytes)